# YOLOV8

### Import Libraries

In [1]:
import os
import cv2
import PIL
import math
import glob
import fiona
import torch
import random
import tifffile
import rasterio
import numpy as np
from PIL import Image
from ultralytics import YOLO
from fiona.crs import from_epsg
import matplotlib.pyplot as plt
from rasterio.features import shapes
from rasterio.transform import from_origin
from shapely.geometry import shape, mapping


### Paths for the snipets

In [2]:
main_folder = "E:\\Jamnagar_24Jun2024" 
main_tif_path = os.path.join(main_folder,"j_jmngr1.tif")
output_subregions_folder = os.path.join(main_folder,"cropped") # Cropped Output
subregion_size=1024

input_dir_containing_jpg = output_subregions_folder # Cropped JPG images
output_dir_containing_jpg = os.path.join(main_folder,"predicted") # Output Binary predicted images
model_path = "D:\\Coding\\Bisag\\photovoltaic-detection-main\\model\\best (8) jodhpur part 200.pt" # path to the Model

merged_predicted_image_path=os.path.join(main_folder,"merged.jpg") # Merged Predicted Image Directory path

border_width = 4

georeferenced_image_path = os.path.join(main_folder,"geo_merged.tif")
# vector_shape_file_path = os.path.join(main_folder,"vector.shp")

pixel_resolution=0.05 #In Meters

In [3]:
PIL.Image.MAX_IMAGE_PIXELS = None

In [4]:
os.makedirs(output_subregions_folder,exist_ok=True)
os.makedirs(output_dir_containing_jpg, exist_ok=True)

### Cutting TIF into chunks

In [5]:
def extract_subregions(input_path, output_folder, subregion_size):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    image = tifffile.imread(input_path)
    height, width = image.shape[:2]
    subregion_count = 1

    for y in range(0, height, subregion_size):
        for x in range(0, width, subregion_size):
            top = y
            left = x

            bottom = min(y + subregion_size, height)
            right = min(x + subregion_size, width)

            subregion = image[top:bottom, left:right]

            # Add padding if necessary
            if bottom - top < subregion_size or right - left < subregion_size:
                padded_subregion = np.zeros((subregion_size, subregion_size, image.shape[2]), dtype=image.dtype)
                padded_subregion[:bottom-top, :right-left] = subregion
                subregion = padded_subregion

            # Save without metadata
            output_path = os.path.join(output_folder, f"{subregion_count}.tif")
            tifffile.imwrite(output_path, subregion, metadata=None)
            subregion_count += 1

### TIF To JPG

In [6]:
def convert_tif_to_jpg(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.tif'):
            tif_path = os.path.join(folder_path, filename)
            with Image.open(tif_path) as img:
                img = img.convert('RGB')
                
                jpg_path = os.path.join(folder_path, os.path.splitext(filename)[0] + '.jpg')
                
                img.save(jpg_path, 'JPEG', quality=100)

### Deleting Unnecessary TIF Files

In [7]:
def delete_tif_files(folder_path):
    # Use glob to find all .tif files in the specified folder
    tif_files = glob.glob(os.path.join(folder_path, "*.tif"))
    
    # Loop through the list of .tif files and delete each one
    for tif_file in tif_files:
        try:
            os.remove(tif_file)
            # print(f"Deleted: {tif_file}")
        except Exception as e:
            print(f"Error deleting {tif_file}: {e}")

### Prediction Code

In [8]:
def predict():
    os.makedirs(output_dir_containing_jpg, exist_ok=True)


    model = YOLO(model_path)

    image_files = [f for f in os.listdir(input_dir_containing_jpg) if os.path.isfile(os.path.join(input_dir_containing_jpg, f)) and f.lower().endswith('.jpg')]

    for image_file in image_files:
        img_path = os.path.join(input_dir_containing_jpg, image_file)
        img = cv2.imread(img_path)

        results = list(model.predict(source=img.copy(), save=False, save_txt=False, stream=True, show_boxes=False, conf=0.2,verbose=False))

        if results[0] is None or len(results[0]) == 0:  # If no detections found
            black_image = np.zeros_like(img)
            output_path = os.path.join(output_dir_containing_jpg, f'{os.path.splitext(image_file)[0]}.jpg')
            cv2.imwrite(output_path, black_image)
        else:
            for result in results:
                masks = result.masks.data
                boxes = result.boxes.data

                clss = boxes[:, 5]

                people_indices = torch.where(clss == 0)

                people_masks = masks[people_indices]

                people_mask = torch.any(people_masks, dim=0).int() * 255

                output_path = os.path.join(output_dir_containing_jpg, f'{os.path.splitext(image_file)[0]}.jpg')    
                cv2.imwrite(output_path, people_mask.cpu().numpy())



### Replace Border Pixels

In [9]:
def replace_border_pixels(image_path, output_path, border_width):
  img = Image.open(image_path)
  width, height = img.size

  # Validate image size and border width
  if border_width * 2 >= width or border_width * 2 >= height:
    raise ValueError(f"Border width ({border_width}) cannot be more than half the image dimensions.")

  pixels = img.load()

  # Replace top pixels
  for x in range(width):
    for y in range(border_width):
      new_pixel_value = pixels[x, border_width + y]
      pixels[x, y] = new_pixel_value

  # Replace bottom pixels (reversed order)
  for x in range(width):
    for y in range(height - border_width, height):
      new_pixel_value = pixels[x, y - border_width - 1]
      pixels[x, y] = new_pixel_value

  # Replace left pixels
  for y in range(height):
    for x in range(border_width):
      new_pixel_value = pixels[border_width + x, y]
      pixels[x, y] = new_pixel_value

  # Replace right pixels (reversed order)
  for y in range(height):
    for x in range(width - border_width, width):
      new_pixel_value = pixels[x - border_width - 1, y]
      pixels[x, y] = new_pixel_value

  # Save the modified image
  img.save(output_path)

def process_folder(folder_path, output_folder_path, border_width):


  for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
      image_path = os.path.join(folder_path, filename)
      output_path = os.path.join(output_folder_path, filename) if output_folder_path else image_path
      try:
        replace_border_pixels(image_path, output_path, border_width)
        # print(f"Processed image: {filename}")
      except ValueError as e:
        print(f"Error processing {filename}: {e}")

### Merge Predicted Images

In [10]:
def get_num_tiles(width, height):
  
  num_tiles_x = math.ceil(width / subregion_size)
  num_tiles_y = math.ceil(height / subregion_size)
  return num_tiles_x, num_tiles_y

In [11]:
def merge_images(image_folder, output_image_path, rows, cols):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')], key=lambda x: int(os.path.splitext(x)[0]))
    image_paths = [os.path.join(image_folder, f) for f in image_files]

    if len(image_paths) < rows * cols:
        raise ValueError("Not enough images to fill the grid.")
    # print(image_paths)

    images = [np.array(Image.open(img_path).convert('RGB')) for img_path in image_paths[:rows * cols]]
    
    img_height, img_width, img_channels = images[0].shape

    merged_image_array = np.zeros((rows * img_height, cols * img_width, img_channels), dtype=np.uint8)

    for index, img in enumerate(images):
        # print(img)
        row = index // cols
        col = index % cols
        merged_image_array[row * img_height:(row + 1) * img_height, col * img_width:(col + 1) * img_width, :] = img

    merged_image = Image.fromarray(merged_image_array)
    
    merged_image.save(output_image_path)

    print(f"Merged image saved to {output_image_path}")


### Crop the merged image

In [12]:
def crop_image(image_path, crop_width, crop_height, output_path=None):
    image = Image.open(image_path)
    width, height = image.size
    crop_area = (0, 0, width - crop_width, height - crop_height)
    cropped_image = image.crop(crop_area)
    if output_path is None:
        output_path = image_path
    cropped_image.save(output_path)
    
    print(f'Cropped image saved as {output_path} with dimensions {cropped_image.size}')



In [13]:
def get_image_size(image_path):
  try:
    # Try Pillow (PIL Fork) first
    from PIL import Image
    img = Image.open(image_path)
    width, height = img.size
    return width, height

  except (OSError, IOError):
    # If Pillow fails, try OpenCV
    import cv2
    img = cv2.imread(image_path)
    if img is None:
      return None
    height, width = img.shape[:2]
    return width, height

  except ImportError:
    # Handle case where neither library is installed
    print("Error: Please install Pillow or OpenCV library.")
    return None

### Geo-Reference the Image

In [14]:
def get_georeferencing_info(tif_path):
    with rasterio.open(tif_path) as dataset:
        transform = dataset.transform
        crs = dataset.crs
    return transform, crs

# Function to georeference a JPG image
def geo_reference_jpg(jpg_path, output_tif_path, transform, crs):
    # Read the JPG image
    img = cv2.imread(jpg_path)

    # Get the dimensions of the image
    height, width, channels = img.shape

    # Create a new rasterio dataset
    new_transform = from_origin(transform.c, transform.f, transform.a, -transform.e)
    with rasterio.open(
        output_tif_path, 'w',
        driver='GTiff',
        height=height,
        width=width,
        count=channels,
        dtype=img.dtype,
        crs=crs,
        transform=new_transform,
    ) as dst:
        for i in range(1, channels + 1):
            dst.write(img[:, :, i - 1], i)

### Create Vector from Georeferenced TIF Image

In [15]:
# def raster_to_vector(georeferenced_image_path, vector_shape_file_path, threshold_value=240):
#     with rasterio.open(georeferenced_image_path) as src:
#         raster = src.read(1)  # Read the first band
#         transform = src.transform
#         raster_crs = src.crs  # Get the CRS of the raster

#     threshold_value = 128
#     binary_raster = (raster > threshold_value).astype(np.uint8)

#     shapes_generator = shapes(binary_raster, transform=transform)
#     polygons = []
#     for geom, value in shapes_generator:
#         if value == 1:
#             polygons.append(shape(geom))

#     output_path = vector_shape_file_path

#     crs_epsg_code = 4326  

#     schema = {
#         'geometry': 'Polygon',
#         'properties': {'id': 'int'},
#     }

#     crs = from_epsg(crs_epsg_code)  
#     with fiona.open(output_path, 'w', driver='ESRI Shapefile', schema=schema, crs=crs) as shp:
#         for i, polygon in enumerate(polygons):
#             shp.write({
#                 'geometry': mapping(polygon),
#                 'properties': {'id': i},
#             })

#     print(f"Vector file saved at: {vector_shape_file_path}")



In [16]:
extract_subregions(main_tif_path, output_subregions_folder, subregion_size)
convert_tif_to_jpg(output_subregions_folder)
delete_tif_files(output_subregions_folder)
predict()
process_folder(output_dir_containing_jpg, output_dir_containing_jpg, border_width)
width_tif, height_tif = get_image_size(main_tif_path)
columns, rows = get_num_tiles(width_tif, height_tif)
merge_images(image_folder=output_dir_containing_jpg, output_image_path=merged_predicted_image_path, rows=rows, cols=columns)
width_jpg, height_jpg = get_image_size(merged_predicted_image_path)
crop_image(merged_predicted_image_path, width_jpg-width_tif, height_jpg-height_tif,merged_predicted_image_path)
transform, crs = get_georeferencing_info(main_tif_path)
geo_reference_jpg(merged_predicted_image_path, georeferenced_image_path, transform, crs)
# raster_to_vector(georeferenced_image_path, vector_shape_file_path, threshold_value=240)


Merged image saved to E:\Jamnagar_24Jun2024\merged.jpg
Cropped image saved as E:\Jamnagar_24Jun2024\merged.jpg with dimensions (27057, 30512)


### Calculate Total Solar Potential Area

In [17]:
# image = cv2.imread(merged_predicted_image_path, cv2.IMREAD_GRAYSCALE)

# _, binary_image = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY)

# image_array = np.array(binary_image)

# white_pixel_count = np.sum(image_array == 255)

# pixel_area_m2 = pixel_resolution*pixel_resolution  # m^2

# total_white_area_m2 = white_pixel_count * pixel_area_m2

# total_white_area_m2 = total_white_area_m2  # 1 m^2 = 10,000 cm^2

# total_white_area_km2 = total_white_area_m2 / 1000000

# print(f'Total number of white pixels: {white_pixel_count}')
# print(f'Total area covered by white pixels: {total_white_area_m2} m^2')
# print(f'Total area covered by white pixels: {total_white_area_km2} km^2')


### Compare 2 Models Code

In [18]:
# image_dir = output_subregions_folder

# model = YOLO("D:\\Coding\\Bisag\\photovoltaic-detection-main\\model\\best (8) jodhpur part.pt")
# model2 = YOLO("D:\\Coding\\Bisag\\photovoltaic-detection-main\\model\\best (8) jodhpur part 200.pt")

# for i in range(int(input())):
#     index_random_sample = random.choice(os.listdir(image_dir))
#     image_path = os.path.join(image_dir, index_random_sample)
    
#     input_image = Image.open(image_path)
    
#     results = model(image_path, conf=0.3, show_boxes=False)
#     results2 = model2(image_path, conf=0.3, show_boxes=False)
    
#     fig = plt.figure()
#     fig.set_size_inches(12, 4)
    
#     ax1 = fig.add_subplot(1, 3, 1)
#     ax1.title.set_text('Input Image')
#     ax1.imshow(input_image)
    
#     ax2 = fig.add_subplot(1, 3, 2)
#     ax2.title.set_text('Predicted Image 1')
#     transformed_image = results[0].plot(boxes=False)
#     ax2.imshow(transformed_image)

#     ax3 = fig.add_subplot(1, 3, 3)
#     ax3.title.set_text('Predicted Image 2')
#     transformed_image = results2[0].plot(boxes=False)
#     cv2.imwrite(image_path, results)  
#     ax3.imshow(transformed_image)
    
#     plt.show()

### Transforming Images

In [19]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# import os

# def process_image(image_path, output_folder):
#     image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

#     if image is None:
#         print(f"The image file at {image_path} was not found or could not be loaded.")
#         return

#     _, binary_image = cv2.threshold(image, 250, 255, cv2.THRESH_BINARY)

#     kernel = np.ones((3, 3), np.uint8)
#     cleaned_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)

#     contours, _ = cv2.findContours(cleaned_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     rotated_rects = [cv2.minAreaRect(contour) for contour in contours]

#     for rotated_rect in rotated_rects:
#         box = cv2.boxPoints(rotated_rect)
#         box = np.int0(box)

#         mask = np.zeros_like(image)
#         cv2.drawContours(mask, [box], 0, (255, 255, 255), -1)

#         image[mask == 255] = 255

#     filename = os.path.basename(image_path)
#     result_path = os.path.join(output_folder, filename)
#     cv2.imwrite(result_path, image)

# input_folder = output_dir_containing_jpg
# output_folder = output_dir_with_transformed_jpg

# os.makedirs(output_folder, exist_ok=True)

# for filename in os.listdir(input_folder):
#     if filename.endswith('.jpg') or filename.endswith('.png'):  # Filter JPEG and PNG files
#         image_path = os.path.join(input_folder, filename)
#         process_image(image_path, output_folder)

### Change the size of black images if model output size is not equal to input size

In [20]:
# import cv2
# import os

# def resize_images(folder_path):
#   for filename in os.listdir(folder_path):
#     image_path = os.path.join(folder_path, filename)
#     if os.path.isfile(image_path) and get_image_size(image_path) == (4096, 4096):
#       resize_image(image_path)

# def get_image_size(image_path):
#   img = cv2.imread(image_path)
#   if img is not None:
#     return img.shape[:2]
#   else:
#     print(f"Error reading image: {image_path}")
#     return None

# def resize_image(image_path):
#   img = cv2.imread(image_path)
#   resized_img = cv2.resize(img, (1024, 1024))
#   cv2.imwrite(image_path, resized_img)

# def delete_tif_files(directory):
#   for root, _, files in os.walk(directory):
#     for file in files:
#       if file.endswith(".tif"):
#         filepath = os.path.join(root, file)
#         os.remove(filepath)
#         print(f"Deleted: {filepath}")


# folder_path = output_dir_containing_jpg

# resize_images(folder_path)
# delete_tif_files(folder_path)
# print("Done")

### Merged Transformed Images

In [21]:
# import cv2
# import os

# def resize_images(folder_path):
#   for filename in os.listdir(folder_path):
#     image_path = os.path.join(folder_path, filename)
#     if os.path.isfile(image_path) and get_image_size(image_path) == (4096, 4096):
#       resize_image(image_path)

# def get_image_size(image_path):
#   img = cv2.imread(image_path)
#   if img is not None:
#     return img.shape[:2]
#   else:
#     print(f"Error reading image: {image_path}")
#     return None

# def resize_image(image_path):
#   img = cv2.imread(image_path)
#   resized_img = cv2.resize(img, (1024, 1024))
#   cv2.imwrite(image_path, resized_img)

# def delete_tif_files(directory):
#   for root, _, files in os.walk(directory):
#     for file in files:
#       if file.endswith(".tif"):
#         filepath = os.path.join(root, file)
#         os.remove(filepath)
#         print(f"Deleted: {filepath}")


# folder_path = output_dir_with_transformed_jpg

# resize_images(folder_path)
# delete_tif_files(folder_path)
# print("Done")

In [22]:
# from PIL import Image
# import numpy as np
# import os

# def merge_images(image_folder, output_image_path, rows, cols):
#     image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')], key=lambda x: int(os.path.splitext(x)[0]))
#     image_paths = [os.path.join(image_folder, f) for f in image_files]

#     if len(image_paths) < rows * cols:
#         raise ValueError("Not enough images to fill the grid.")

#     images = [np.array(Image.open(img_path).convert('RGB')) for img_path in image_paths[:rows * cols]]
    
#     img_height, img_width, img_channels = images[0].shape

#     merged_image_array = np.zeros((rows * img_height, cols * img_width, img_channels), dtype=np.uint8)

#     for index, img in enumerate(images):
#         row = index // cols
#         col = index % cols
#         merged_image_array[row * img_height:(row + 1) * img_height, col * img_width:(col + 1) * img_width, :] = img

#     merged_image = Image.fromarray(merged_image_array)
    
#     merged_image.save(output_image_path)
#     print(f"Merged image saved to {output_image_path}")

# merge_images(image_folder=output_dir_with_transformed_jpg, output_image_path=merged_transformed_image_path, rows=rows, cols=columns)
